<a href="https://colab.research.google.com/github/satoukazunobu/2311-conflict-practice/blob/main/SD4_HB_Section04_02_10_%E4%BD%90%E8%97%A4%E5%92%8C%E4%BC%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 高度プログラミング演習Ｂ プログラムテンプレート
## 統一指示


1.   以下の形式でファイル名の「出席番号_フルネーム」の箇所を自分にあわせて変更してください
2.   メニューの[ファイル]->[ドライブにコピーを保存]から自分のGoogleDrive内にコピーを保存してから、編集してください。最後に指示があったら、メニューの[ファイル]->[Githubにコピーを保存]からGithubOrganizationの25SD4組織内にコピーを保存・提出してください。





In [1]:
!pip install -U langchain langchain-google-genai google-generativeai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
import os

os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

In [3]:
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")

In [4]:
from langchain.prompts import PromptTemplate
# {dish}という変数が含まれるテンプレートを作成
prompt = PromptTemplate.from_template("""
    以下の料理のレシピを作成してください。
    料理名：{dish}
 """
 )
# invokeメソッドで変数{dish}の値を差し込む
prompt_value = prompt.invoke({"dish":"カレー"})

print(prompt_value.to_string())



    以下の料理のレシピを作成してください。
    料理名：カレー
 


In [5]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "ユーザーが入力した料理のレシピを考えてください。"),
    ("human", "{dish}"), # humanはuserロール相当
])

prompt_value = prompt.invoke({"dish":"カレー"})
print(prompt_value)

messages=[SystemMessage(content='ユーザーが入力した料理のレシピを考えてください。', additional_kwargs={}, response_metadata={}), HumanMessage(content='カレー', additional_kwargs={}, response_metadata={})]


In [6]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system","You are helpful assistant."),
    # humanとAIの会話履歴を、MessagesPlaceholderオブジェクトで埋め込む
    MessagesPlaceholder("chat_history", optional=True),
    # 履歴じゃない追加のメッセージ
    ("human","{input}"),
])

#promptに組み込まれたテンプレートに値をはめ込んで会話を投げる
prompt_value = prompt.invoke({
    "chat_history":[HumanMessage(content="こんにちは！　私はジョンと言います！"),
                    AIMessage(content="こんにちは、ジョンさん！　どのようにお手伝いできますか？",)
                    ],
    "input":"私の名前がわかりますか？"
})
print(prompt_value)

messages=[SystemMessage(content='You are helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='こんにちは！\u3000私はジョンと言います！', additional_kwargs={}, response_metadata={}), AIMessage(content='こんにちは、ジョンさん！\u3000どのようにお手伝いできますか？', additional_kwargs={}, response_metadata={}), HumanMessage(content='私の名前がわかりますか？', additional_kwargs={}, response_metadata={})]


In [7]:
# langchainhubをimportする
!pip install langchainhub -q
from langchain import hub

# 公開されているプロンプトを取得
prompt = hub.pull("oshima/recipe") #oshimaは教科書の作者

# 通常のChatPromptTemplateと同様に使える
prompt_value = prompt.invoke({"dish":"カレー"})
print(prompt_value)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.2 MB/s eta 0:00:00
messages=[SystemMessage(content='ユーザーが入力した料理のレシピを考えてください。', additional_kwargs={}, response_metadata={}), HumanMessage(content='カレー', additional_kwargs={}, response_metadata={})]


In [8]:
from pydantic import BaseModel, Field
from typing import List

class Recipe(BaseModel):
  ingredients:List[str] = Field(description="料理の材料リスト")
  steps:List[str] = Field(description="調理手順のリスト")

In [9]:
from langchain_core.output_parsers import PydanticOutputParser

output_parser = PydanticOutputParser(pydantic_object=Recipe)

format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "料理の材料リスト", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "調理手順のリスト", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```


In [16]:
#プロンプトを作る
prompt = ChatPromptTemplate.from_messages([
    ("system","ユーザーが入力した料理のレシピを考えてください。\n\n{format_instructions}"),
    ("human","{dish}"),
])
#プロンプトに形式の指示「format_instructions」を埋め込む
prompt_with_parser = prompt.partial(format_instructions=format_instructions)


In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-flash-latest", google_api_key=GEMINI_API_KEY,temperatur=0.7)

#modelにRecipeクラスの情報を渡す
structured_llm = model.with_structured_output(Recipe)

#プロンプトとモデルを結合(LCEL)
chain = prompt_with_parser | structured_llm

#プロンプトとモデルの処理がつながっているchainを使ってメッセージを投げる
recipe_object = chain.invoke({"dish":"ハンバーグ"})

print(type(recipe_object))
print(recipe_object)

<class '__main__.Recipe'>
ingredients=['合いびき肉: 300g', '玉ねぎ: 1/2個', 'パン粉: 大さじ3', '牛乳: 大さじ3', '卵: 1個', '塩: 小さじ1/2', 'こしょう: 少々', 'ナツメグ: 少々', 'サラダ油: 大さじ1 (焼く用)', '【ソース】ケチャップ: 大さじ3', '【ソース】ウスターソース: 大さじ3', '【ソース】水または赤ワイン: 大さじ1'] steps=['玉ねぎをみじん切りにし、フライパンで透明になるまで炒めて冷まします。', 'パン粉を牛乳に浸しておきます。', 'ボウルに合いびき肉、冷ました玉ねぎ、パン粉と牛乳、卵、塩、こしょう、ナツメグを入れ、粘り気が出るまでよくこねます。', 'タネを2～3等分にし、空気を抜きながら小判型に成形します。真ん中を少しくぼませるのがポイントです。', 'フライパンにサラダ油をひき中火で熱し、ハンバーグを並べます。焼き色がついたら裏返します。', '蓋をして弱火で5～8分蒸し焼きにします。竹串を刺して透明な肉汁が出れば火が通っています。', 'ハンバーグを皿に取り出します。', '同じフライパンにソースの材料（ケチャップ、ウスターソース、水または赤ワイン）を入れ、軽く煮詰めてソースを作ります。', 'ハンバーグにソースをかけて完成です。']
